Try nmf with all non negative features with grid search of c 


In [1]:
import pandas as pd


import matplotlib.pyplot as plt
from sklearn.decomposition import NMF 
from sklearn.preprocessing import Normalizer ,MaxAbsScaler
from sklearn.pipeline import make_pipeline

In [2]:
df = pd.read_csv('../data/final_per_90_and_pAdj.csv')

In [3]:
df.loc['Walker Zimmerman']

KeyError: 'Walker Zimmerman'

In [3]:
# ## dont need club for NMF 

clubs = df['Club']
df.drop(columns=['Club'],inplace=True)
missing = [df.isnull().sum(),100*df.isnull().mean()]
m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()
df['Base Salary'] = df['Base Salary'].str.replace('$','')
df['Base Salary'] = df['Base Salary'].str.replace(',','')
df['Base Salary'] = pd.to_numeric(df['Base Salary'])
df[['Base Salary']].info()
cols = ['Base Salary']
for col in cols:
    print(df[col].mean())
    df[col] = df[col].fillna(df[col].mean())

missing = [df.isnull().sum(),100*df.isnull().mean()]
m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Base Salary  521 non-null    float64
dtypes: float64(1)
memory usage: 5.6 KB
381064.81381957774


<ipython-input-3-1ce0fdadc409>:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Base Salary'] = df['Base Salary'].str.replace('$','')


,count,%
True_tackle_pct,33.0,4.680851
fb_id,0.0,0.000000
PKatt,0.0,0.000000
pAdj_Tackles_Mid_3rd,0.0,0.000000
pAdj_Tackles_Def_3rd,0.0,0.000000


In [13]:
#drop cols with negatives 

df_n = df.drop(columns= ['True_tackle_pct','Player', 'Nation', 'Position','fb_id', 'player_name', 'Squad_from_op','G-xG','np:G-xG'])
df_n.shape

(705, 96)

In [10]:
df_n.columns.values

array(['fb_id', 'player_name', 'Squad_from_op', 'Opposing_Poss',
       'Opposing_Touches', 'Player', 'Nation', 'Position',
       'Tackle_pct_won', 'Nineties', 'xA', 'Key_pass', 'Comp_prog',
       'Total_Cmp', 'Total_Att', 'Total_Cmp_pct', 'Total_Tot_Dist',
       'Total_Prg_Dist', 'Short_Cmp', 'Short_Att', 'Short_Cmp_pct',
       'Medium_Cmp', 'Medium_Att', 'Medium_Cmp_pct', 'Long_Cmp',
       'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
       'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
       'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
       'dribble_Succ_pct', 'num_dribble_past', 'dribble_megs', 'carries',
       'carries_dist', 'carries_prog_dist', 'carries_prog',
       'carries_att_3rd', 'carries_att_box', 'miss_control', 'dispossed',
       'recep_targ', 'recep_succ', 'recept_pct', 'Fouls_drawn',
       'Offsides', 'Crosses', 'PK_won', 'Aerial_win_pct', 'Gls', 'Sh',
       'SoT', 'SoT%', 'Sh/90', 'SoT/90', 'G/Sh', 'G/SoT', 'Dist'

In [ ]:
transformer = MaxAbsScaler().fit(X)

scaled_X= transformer.transform(X)

transformer = Normalizer().fit(scaled_X)
normed_X= transformer.transform(scaled_X)

In [17]:

errors = {}
components_dict = {}
for c in range(4,12):
    X = df_n  # reading the score data
    
    c = c
    # Create a MaxAbsScaler: scaler
    transformer = MaxAbsScaler().fit(X)

    scaled_X= transformer.transform(X)
    
    # Create an NMF model: nmf
    
    transformer = Normalizer().fit(scaled_X)
    normed_X= transformer.transform(scaled_X)
    nmf = NMF(n_components=c,max_iter=100000,init='nndsvda', random_state=0)
    print(nmf)


    W = nmf.fit_transform(normed_X)
    H = nmf.components_
    err = nmf.reconstruction_err_
    errors[c]=err
    # Create a DataFrame: components_df
    components_df = pd.DataFrame(nmf.components_,columns=df_n.columns)
    # put data frame in dictionary to observe 
    components_dict[c]=components_df


NMF(init='nndsvda', max_iter=100000, n_components=4, random_state=0)
NMF(init='nndsvda', max_iter=100000, n_components=5, random_state=0)
NMF(init='nndsvda', max_iter=100000, n_components=6, random_state=0)
NMF(init='nndsvda', max_iter=100000, n_components=7, random_state=0)
NMF(init='nndsvda', max_iter=100000, n_components=8, random_state=0)
NMF(init='nndsvda', max_iter=100000, n_components=9, random_state=0)
NMF(init='nndsvda', max_iter=100000, n_components=10, random_state=0)
NMF(init='nndsvda', max_iter=100000, n_components=11, random_state=0)


# looking for coherent groups of features inside of components to pick c  

In [18]:
for i in components_dict:
    print('This is nmf with :',i, ' components')
    print(' error = ',errors[i])
    for a in range(len(components_dict[i])):
        component = components_dict[i].iloc[a]
        print(component.nlargest())
    print('\n')

This is nmf with : 4  components
 error =  7.07055002060699
Opposing_Poss       1.276395
Opposing_Touches    1.256978
Short_Cmp_pct       1.168685
Medium_Cmp_pct      1.081329
Total_Cmp_pct       1.055303
Name: 0, dtype: float64
Opposing_Poss       0.962283
Opposing_Touches    0.961859
Short_Cmp_pct       0.847617
Total_Cmp_pct       0.758826
Medium_Cmp_pct      0.756652
Name: 1, dtype: float64
Short_Cmp_pct       0.750839
Medium_Cmp_pct      0.750383
recept_pct          0.747014
Opposing_Poss       0.725815
Opposing_Touches    0.720652
Name: 2, dtype: float64
Medium_Att    0.898526
Total_Att     0.888954
Medium_Cmp    0.858334
Total_Cmp     0.851128
touch_live    0.839981
Name: 3, dtype: float64


This is nmf with : 5  components
 error =  6.690215354760333
Opposing_Poss          1.229818
Opposing_Touches       1.200414
Short_Cmp_pct          1.135282
pAdj_total_pressing    1.122464
Medium_Cmp_pct         1.053617
Name: 0, dtype: float64
Opposing_Poss       1.095788
Opposing_Touches  

In [19]:

X = df_n  # reading the score data

c = 9
# Create a MaxAbsScaler: scaler
transformer = MaxAbsScaler().fit(X)

scaled_X= transformer.transform(X)

# Create an NMF model: nmf

nmf = NMF(n_components=c,max_iter=100000,init='nndsvda', random_state=0)
print(nmf)


W = nmf.fit_transform(scaled_X)
H = nmf.components_
err = nmf.reconstruction_err_

# Create a DataFrame: components_df
components_df = pd.DataFrame(nmf.components_,columns=df_n.columns)


NMF(init='nndsvda', max_iter=100000, n_components=9, random_state=0)


In [20]:
err

19.742031406115526

In [33]:
for i in range(len(components_df)):
    component = components_df.iloc[i]
    print(component.nlargest())

Opposing_Poss       7.120503
Opposing_Touches    7.047423
Short_Cmp_pct       6.849966
Medium_Cmp_pct      6.469437
Total_Cmp_pct       6.227953
Name: 0, dtype: float64
SoT%                3.132418
Offsides            3.026623
Opposing_Touches    2.870311
Opposing_Poss       2.869142
Sh                  2.843840
Name: 1, dtype: float64
Opposing_Poss             1.195030
pAdj_Tackles_Def_3rd      1.129401
Opposing_Touches          1.125644
pAdj_Blocked_pass         1.100131
pAdj_Tackle_int_blocks    1.064466
Name: 2, dtype: float64
Total_Att         1.305865
Total_Cmp         1.274991
recep_succ        1.241687
Total_Tot_Dist    1.164056
recep_targ        1.143507
Name: 3, dtype: float64
Nineties            4.028830
Opposing_Poss       1.867471
Opposing_Touches    1.805198
Short_Cmp_pct       1.710985
Medium_Cmp_pct      1.647035
Name: 4, dtype: float64
dribble_Succ        1.787606
num_dribble_past    1.665060
dispossed           1.331048
dribble_Att         1.245194
carries_att_box    

In [34]:
transformer = Normalizer().fit(W)
normed_X= transformer.transform(W)
normed_df = pd.DataFrame(normed_X,index=df.player_name.values)
normed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 705 entries, Carles Gil to Kamron Habibullah
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       705 non-null    float64
 1   1       705 non-null    float64
 2   2       705 non-null    float64
 3   3       705 non-null    float64
 4   4       705 non-null    float64
 5   5       705 non-null    float64
 6   6       705 non-null    float64
 7   7       705 non-null    float64
 8   8       705 non-null    float64
dtypes: float64(9)
memory usage: 55.1+ KB


In [35]:
normed_df.head(3)

,0,1,2,3,4,5,6,7,8
Carles Gil,0.0,0.067262,0.073429,0.725953,0.322926,0.570075,0.00000,0.183872,0.000000
Julian Gressel,0.0,0.075672,0.536322,0.629231,0.400568,0.236684,0.00000,0.268730,0.148361
Emanuel Reynoso,0.0,0.073697,0.194931,0.483902,0.253861,0.714135,0.38069,0.055229,0.000000


In [37]:
tt = normed_df.loc['Tommy Thompson']
sims = normed_df.dot(tt)

sims.nlargest(10)

Tommy Thompson     1.000000
Jaylin Lindsey     0.988892
Oniel Fisher       0.985663
Emmanuel Más       0.983475
Tayvon Gray        0.978975
Niko Hämäläinen    0.970592
Eddie Munjoma      0.969803
D.J. Taylor        0.969100
Zorhan Bassong     0.967453
Edgar Castillo     0.964880
dtype: float64

In [26]:
print([i for i in df['player_name'] if 'Walker' in i ])

['Walker Zimmerman']


In [27]:
wz = normed_df.loc['Walker Zimmerman']
sims = normed_df.dot(wz)

sims.nlargest(10)

Walker Zimmerman     1.000000
Tanner Beason        0.994719
Bill Tuiloma         0.994329
Ranko Veselinović    0.993640
Xavier Arreaga       0.990380
Jonathan Mensah      0.989931
Andy Rose            0.989094
Rudy Camacho         0.988755
Robin Jansson        0.987397
Oswaldo Alanís       0.987071
dtype: float64

In [28]:
jm = normed_df.loc['Jonathan Mensah']
sims = normed_df.dot(jm)

sims.nlargest(10)

Jonathan Mensah      1.000000
Andy Rose            0.993976
Antonio Carlos       0.993851
Rudy Camacho         0.993287
Matt Hedges          0.991502
Julio Cascante       0.991042
Larrys Mabiala       0.990534
Walker Zimmerman     0.989931
Ranko Veselinović    0.989466
Josh Williams        0.988189
dtype: float64

## need to test results with less features to see if error goes down 